# Analysis of comments in three FCC dockets

This notebook contains Python code that runs the following steps:

- Loading the four comment datasets under analysis (published comments for FCC dockets 14-28, 16-42, 17-108, plus bulk-uploaded comments for docket 17-108).


- Classifying the comments for dockets 14-28 and 16-42, based on the language used in them.


- Examining:

    - How often email addresses the 17-108 bulk uploads appear in data breaches identified by [Have I Been Pwned](https://haveibeenpwned.com/)

    - The overlap between comments in docket 16-42 and bulk-uploaded comments in docket 17-108

    - The comments attributed to Annie Reeves vis-a-vis the timing and language used in American Commitment's docket 14-28 and docket 16-42 mass-comment campaigns.
    
__Please see this repository's landing page and associated BuzzFeed News article (linked on the landing page) for context before continuing.__

---

# Import Python libraries and set key variables

In [1]:
# Standard libraries
import os
import sys
import time
import re

# External libraries
import pandas as pd

In [2]:
# Change this to True if you plan to reuse this notebook
# and want to make HTTP requests to Have I Been Pwned's API

MAKE_HTTP_REQUESTS = False

if MAKE_HTTP_REQUESTS:
    from tqdm.auto import tqdm
    import requests
    import requests_cache
    
    # This is the API key for Have I Been Pwned
    HIBP_KEY = open("../hibp-key.txt").read().strip()
    
    # Enables graphical progress bars when fetching HIPB data
    tqdm.pandas()
    
    # For caching HTTP requests
    requests_cache.install_cache(
        "../hibp-requests-cache",
        allowable_codes = (200, 404),
    )

In [3]:
BASE_PATH = "../data/"

# In the sampling procedures below, we use this "random state"
# to make the samples reproducible. 
RANDOM_STATE = 0

# Load comments

In [4]:
def load_comments(path, **kwargs):
    return (
        pd.read_csv(
            path,
            dtype = str,
            **kwargs
        )
        .astype({
            "email_address_nonstandard": int
        })
    )

### Docket 17-108, bulk uploads

In [5]:
bulk_uploads_17_108 = load_comments(BASE_PATH + "bulk-uploads-17-108-with-uuids.csv")

bulk_uploads_17_108.head()

,date,comments,file,uploader,email_address_nonstandard,email_address,email_domain,name_and_location
0,5/8/2017,"Dear FCC, I am am writing today to SUPPORT net...",ecfs-input-template-17-108 (209).csv,kathleenkintz@gmail.com,0,9f664e24-96aa-4d96-b453-24d926658b47,gmail.com,5100f64f-b025-467f-9aa6-0100fa615ae6
1,12/31/2017,"Dear FCC, I am writing you today because I spe...",ecfs-input-template-17-108 (120).csv,vgboy522@gmail.com,0,818761bf-4c51-4970-95e6-11b01bac631f,gmail.com,dda3bd6b-9ad2-42d0-af15-f12c0b8a9354
2,5/16/17,Obama's Federal Communications Commission (FCC...,TPA_3911_2017526.csv,esmisc@mac.com,0,f2cf802f-0c01-4d1f-b28f-0efef2a053ba,hotmail.com,d9b96c36-796e-45d1-97d8-00647ae09d89
3,5/16/17,Obama's Federal Communications Commission (FCC...,TPA_3911_2017526.csv,esmisc@mac.com,0,6966ae39-6da6-4a47-a1ec-7dc854030634,gmail.com,f6d75f39-e952-41ff-b7a9-3d86da811496
4,5/16/17,Obama's Federal Communications Commission (FCC...,TPA_3911_2017526.csv,esmisc@mac.com,0,610afa24-f0df-44ff-b621-f545d371efab,gmail.com,1b3050d5-6f3a-495e-a67f-b3b61040fe02


#### Examine bulk-uploader metrics for 17-108

In [6]:
uploader_metrics = (
    bulk_uploads_17_108
    .assign(
        prop_with_email = lambda df: df["email_address"].notnull()
    )
    .groupby("uploader")
    .pipe(lambda grp: pd.DataFrame({
        "submissions": grp.size(),
        "unique_emails": grp["email_address"].nunique(),        
        "prop_with_email": grp["prop_with_email"].mean().round(4),
    }))
)
    
(
    uploader_metrics
    .sort_values("submissions", ascending = False)
    .loc[lambda df: df["submissions"] >= 10000]
)

,submissions,unique_emails,prop_with_email
uploader,,,
esmisc@mac.com,4347979,3966016,1.0000
shane@mediabridgellc.com,1856553,1501145,1.0000
mike@fightforthefuture.org,1464423,129682,0.2464
karen@momsrising.org,1069368,17870,0.0362
dutch@freepress.net,528607,3,0.0000
kurt@demandprogress.org,412792,290372,1.0000
fccfreedom@hmamail.com,207007,122252,1.0000
advocacy@mozilla.com,82926,0,0.0000
action@aclu.org,48733,0,0.0000


### Docket 17-108, all comments

In [7]:
comments_17_108 = (
    load_comments(BASE_PATH + "comments-17-108-with-uuids.csv")
    .assign(date = lambda df: df["date"].str.slice(0, 10))
)

comments_17_108.head()

,id_submission,date,email_address_nonstandard,email_address,email_domain
0,04272972619149,2017-04-27,0,NaN,NaN
1,0427547924954,2017-04-27,0,NaN,NaN
2,10427918117987,2017-04-27,0,1f4830aa-726c-4206-9bef-cb3f2a57bb20,gmail.com
3,10427080530667,2017-04-27,0,f10d9c2b-2c98-44c2-9c7a-fe57b96930d8,gmail.com
4,1042709110034,2017-04-27,0,a6609a29-4b4c-4857-9e42-a886f61b8aaa,d6b158e4-d116-4944-ab88-73091f1fc465


#### Examine email domains attributed to these comments

In [8]:
email_domains_17_108 = (
    comments_17_108
    .groupby([ "email_domain" ])
    .pipe(lambda grp: pd.DataFrame({
        "count": grp.size(),
        "unique_addresses": grp["email_address"].nunique()
    }))
    .sort_values([ "count", "unique_addresses" ], ascending = False)
)

(
    email_domains_17_108
    .loc[lambda df: df["unique_addresses"] >= 10000]
    .sort_values("unique_addresses", ascending = False)
)

,count,unique_addresses
email_domain,,
gmail.com,5000687,4160788
yahoo.com,2536892,2126544
hotmail.com,673018,571156
aol.com,632971,508087
pornhub.com,1030003,233516
comcast.net,208512,158939
icloud.com,106442,91091
msn.com,110056,89398
hurra.de,363357,88571


Here we count the comments and unique email addresses associated with FakeMailGenerator.com:

In [9]:
FAKEMAIL_DOMAINS = [
    "einrot.com",
    "jourrapide.com",
    "armyspy.com",
    "fleckens.hu",
    "cuvox.de",
    "rhyta.com",
    "dayrep.com",
    "gustr.com",
    "superrito.com",
    "teleworm.us",
]

In [10]:
(
    email_domains_17_108
    .loc[FAKEMAIL_DOMAINS]
    [["count", "unique_addresses"]]
    .sum()
    .to_frame("count")
)

,count
count,7754231
unique_addresses,167460


### Docket 16-42

In [11]:
comments_16_42 = (
    load_comments(BASE_PATH + "comments-16-42-with-uuids.csv")
    .assign(date = lambda df: df["date"].str.slice(0, 10))
)    

comments_16_42.head()

,date,id_submission,comments,email_address_nonstandard,email_address,email_domain,name_and_location
0,2016-02-19,60001483702,60001515146.txtThank you! Very pleased to see...,0,NaN,NaN,8ad10c4e-1354-42ba-83f1-be6b3c89f331
1,2016-02-22,60001484317,60001843102.txt[5/23/2016 7:55:30 PM]The excha...,0,310d8308-43a0-4b84-93dc-6662acdef829,gmail.com,d984ccab-11bb-4994-bcfe-f0d407fd03b5
2,2016-02-25,60001486876,60001518518.txtPlease eliminate the cable TV b...,0,7e6087df-a7a2-414f-8ebb-3be229805bec,yahoo.com,298bb4d9-8130-4ced-86e9-6a5d0c740c66
3,2016-02-27,60001489444,I?support?the?FCC?allowing?homeowners?to?be?fr...,0,NaN,NaN,12614861-1a8f-4313-aeff-2366bcf18ca8
4,2016-02-29,60001492083,"60001523826.txtAs a consumer, I agree with the...",0,NaN,NaN,10268573-9386-42c7-ab31-4d76641e76ed


### Docket 14-28

In [12]:
comments_14_28 = (
    load_comments(BASE_PATH + "comments-14-28-with-uuids.csv")
    .assign(date = lambda df: df["date"].str.slice(0, 10))
)    

comments_14_28.head()

,date,id_submission,comments,email_address_nonstandard,email_address,email_domain,name_and_location
0,2014-02-21,6017589853,7521074305.txt Reclassify The Internet As A Co...,0,NaN,NaN,a0fad65b-1482-427d-b300-da8e63d14272
1,2014-02-21,6017589866,7521074318.txt Reclassify The Internet As A Co...,0,NaN,NaN,45954c7c-d52d-48f0-a252-343b4f82e509
2,2014-02-21,6017589903,7521074355.txt Reclassify The Internet As A Co...,0,NaN,NaN,0a1dea8a-3ae6-434f-be03-4b6447c3190c
3,2014-02-21,6017589904,7521074356.txt Reclassify The Internet As A Co...,0,NaN,NaN,5cb3b14f-71a4-4763-8bd1-40ad440c5eb8
4,2014-02-21,6017589924,7521074376.txt Reclassify The Internet As A Co...,0,NaN,NaN,0de47b78-a256-4e36-93ad-1f4830b07c48


# Classify comments

In this step, we create derivative dataframes that classify each comment based on the language used in them. (Note: Because the formatting of comments can be inconsistent, the classification approach ignores whitespace.)

The classifier takes a series of texts and a series of patterns to look for. Each text is labeled based on the __first__ pattern it matches, based on the sequential order of the patterns; if the text matches no pattern, it is labeled `[other]`.

In [13]:
def classify(texts, patterns):
    # Create a copy of the texts and remove whitespace
    s = texts.copy().str.replace(r"\s+", "")
    
    # Remove whitespace from classification patterns
    without_whitespace = [ (re.sub(r"\s+", "", pat), val)
        for pat, val in patterns ]
    
    # An empty series, indexed identically to the original texts.
    ix = pd.Series(None, index = texts.index)

    # As we progress through the matching, we will gradually
    # fill `ix` in with the matches we've found.
    
    # Iterate through the classification patterns
    for pat, val in without_whitespace:
        # Determine which texts match
        search_result = s.str.contains(pat, na = False)
        matches = search_result.loc[lambda x: x == True]
        
        # For matches, update `ix` to indicate the pattern ID/description
        ix.loc[matches.index] = val
        
        # Subset `s` so that it only contains unmatched texts
        s = s.loc[s.index.difference(matches.index)]

    return ix.fillna("[other]")

In [14]:
def add_classification(df, patterns):
    return (
        df
        .assign(group = lambda df: (
            df["comments"]
            .pipe(classify, patterns)
        ))
    )

In [15]:
def print_example_comments(df, n = 3, max_chars = 500):
    for grp, subdf in df.groupby("group"):
        print(f"=== {grp} ===\n")
        
        examples = (
            subdf["comments"]
            .sample(n, random_state = RANDOM_STATE)
            .pipe(lambda x: pd.np.where(
                x.apply(len) > max_chars,
                x.str.slice(0, max_chars) + "[...]",
                x
            ))
        )
        
        print("\n\n".join(examples) + "\n\n")

### Docket 14-28

BuzzFeed News identified the phrases below based on extensive examination of the 14-28 docket, and by cross-referencing them with [this December 2014 Sunlight Foundation analysis](http://web.archive.org/web/20150301070951/http://sunlightfoundation.com/blog/2014/12/16/one-group-dominates-the-second-round-of-net-neutrality-comments/).

The `AC-` comments use language from American Commitment's comment campaign. It is possible that entities other than American Commitment submitted comments that used the same language. 

Note: The final phrase in the list below also appears alongside some of the other permutations; but because it is the final phrase in the list, only comments that don't match the other phrasings receive this classification.

Please see this repository's landing page, and the associated BuzzFeed News article, for additional context. (E.g., not all comments are indivudally retreivable from the FCC's public portal.)

In [16]:
ac_patterns_main = [
    # Earlier set of comments
    "The federal government can use their power over the internet to direct content", 
    "because of high barriers to entry and a disastrous lack of competition", 
    "federal bureaucrats will slow down the process and protect prevailing interests first", 
    "blossoming in America today, largely due to the internet", 
    "Government will make it impossible for internet providers to upgrade service", 
    "government will naturally favor entrenched special interests, rather than upstart companies", 
    "will begin to be mismanaged, like many other government-run industries", 
    "put directly in the hands of bureaucrats, instead of the free market", 
    "will result in worse service, even as costs continue to skyrocket", 
    "Our options for cheap, high-speed, high-performing internet providers", 

    # Later set of comments
    "Left-wing extremists have been crying wolf", 
    "The federal government needs to keep its hands off the Internet", 
    "Before the FCC places regulatory handcuffs on Internet providers", 
    "The notion that the internet is broken and needs repair is simply not true", 
    "will send the crown jewel of the US economy into an economic tailspin", 
    "no longer acting in the interests of the American people", 
    "just another slow-moving government-controlled mess", 
    "defend ourselves against power-hungry bureaucrats", 
    "simply another attempt by the federal government to take control of another sector of the economy", 
    "Millions of liberal fools demanding you reduce the Internet", 
    "FCC is clearly ignoring the will of the American people", 
    "devastate private investment with the force of an atomic bomb", 
    "without being slowed by bureaucratic inertia", 
    "A small fringe of the extremist left has been demanding", 
    "a tiny minority of far-left political activists", 
    "ultimate goal is to get rid of the media capitalists", 
    "created economic and human wreckage in their wake", 
    "increase its own power at the expense of the free people", 
    "it will have proven itself to be an unaccountable agency", 
    "subjecting it to 1930s-style regulations meant for telephone monopolies", 
    "Government regulation of Internet services would chase investment", 
    "it will seriously degrade the Internet we have", 
    "it can and should suffer the consequences", 
    "taking such reckless actions to gain control over the Internet", 
    "simply is no evidence to back up the dire claims of disaster", 
]

ac_patterns_other = [
    "Like many Americans, I believe that the internet should remain free of government",
    "As an American citizen, I wanted to voice my opposition to the FCC",
]

ac_pattern_desciptions = (
    [ (p, f"AC-{i:02d}") for i, p in enumerate(ac_patterns_main) ] +
    [ (p, f"AC-other") for p in ac_patterns_other ]
)

Notes:

- The `AC-XX` classification names below are based simply on the order in which they appear above. The numbers have no independent meaning.

- The `AC-other` classification indicates that key language (the two phrases in `ac_patterns_other` above) from American Commitment appears in the comment, but not any of the other phrases.

In [17]:
comments_14_28_classified = (
    comments_14_28
    .pipe(
        add_classification,
        ac_pattern_desciptions
    )
)

(
    comments_14_28_classified
    ["group"]
    .value_counts()
    .sort_index()
)

AC-00          1261
AC-01          1233
AC-02          1246
AC-03          1232
AC-04          1269
AC-05          1208
AC-06          1210
AC-07          1207
AC-08          1202
AC-09          1186
AC-10         25801
AC-11         25781
AC-12         25951
AC-13         26012
AC-14         25667
AC-15         25879
AC-16         25727
AC-17         26009
AC-18         25658
AC-19         25788
AC-20         25924
AC-21         25914
AC-22         25950
AC-23         25865
AC-24         25864
AC-25         25620
AC-26         26044
AC-27         25932
AC-28         25745
AC-29         26024
AC-30         25624
AC-31         25880
AC-32         25691
AC-33         25615
AC-34         25836
AC-other          6
[other]     1396620
Name: group, dtype: int64

The total number of comments and unique email addreses for all `AC-`-classified comments above:

In [18]:
(
    comments_14_28_classified
    .loc[lambda df: df["group"] != "[other]"]
    .pipe(lambda df: pd.Series({
        "comments": len(df),
        "unique_email_addresses": df["email_address"].nunique()
    }))
    .to_frame("count")
)

,count
comments,658061
unique_email_addresses,551855


#### Dates submitted

The analysis below demonstrates that the following:

- Comments `AC-00`-`AC-09` share a similar distribution of dates submitted
- Comments `AC-10`-`AC-34` also share a similar distribution of dates submitted, but distinct from `AC-00`-`AC-09`

Additional notes:

- Dates below are `MM-DD`, for 2014

- Dates include only those with at least 200 total `AC-` classified comments (overall), to reduce noise of stray dates that contain relatively few matching comments

In [19]:
(
    comments_14_28_classified
    .loc[lambda df: df["group"] != "[other]"]
    .assign(
        date = lambda df: df["date"].str.slice(5, 10)
    )
    .groupby(["group", "date"])
    .size()
    .unstack()
    .fillna(0)
    .astype(int)
    .loc[:, lambda df: df.sum() >= 200]
    
    # Order columns by date
    .pipe(lambda df: df[[c for c in sorted(df.columns)]])
    
)

date,07-14,07-16,07-17,09-11,09-12,09-13,09-14,09-15,09-16,09-17,09-18,09-19,09-22,09-23,09-24
group,,,,,,,,,,,,,,,
AC-00,45,909,305,0,0,0,0,0,0,0,0,0,0,0,0
AC-01,39,905,287,0,0,0,0,0,0,0,0,0,0,0,0
AC-02,52,888,304,0,0,0,0,0,0,0,0,0,0,0,0
AC-03,45,893,293,0,0,0,0,0,0,0,0,0,0,0,0
AC-04,80,902,284,0,0,0,0,0,0,0,0,0,0,0,0
AC-05,36,872,300,0,0,0,0,0,0,0,0,0,0,0,0
AC-06,47,906,257,0,0,0,0,0,0,0,0,0,0,0,0
AC-07,48,859,299,0,0,0,0,0,0,0,0,0,0,0,0
AC-08,45,854,303,0,0,0,0,0,0,0,0,0,0,0,0


#### Example comments

In [20]:
print_example_comments(
    comments_14_28_classified
    .loc[lambda df: df["group"].isin(["AC-10", "AC-27", "[other]"])],
)

=== AC-10 ===

The Internet is not broken, and does not need to be fixed.  Left-wing extremists have been crying wolf for the past decade about the harm to the Internet if the Federal government didn?t regulate it.  Not only were they wrong, but the Internet has exploded with innovation.  Do not regulate the Internet.  The best way to keep it open and free is what has kept it open and free all along ? no government intervention.

The Internet is not broken, and does not need to be fixed.  Left-wing extremists have been crying wolf for the past decade about the harm to the Internet if the Federal government didn?t regulate it.  Not only were they wrong, but the Internet has exploded with innovation.  Do not regulate the Internet.  The best way to keep it open and free is what has kept it open and free all along ? no government intervention.

The Internet is not broken, and does not need to be fixed.  Left-wing extremists have been crying wolf for the past decade about the harm to the In

Compare the above timing and language for group `AC-27` to the [comment attributed to Annie Reeves](https://www.fcc.gov/ecfs/filing/6019076835), received by the FCC on September 15, 2014:

> The American people are watching a Federal Communications Commission (FCC) that is not seeking to make the Internet better, but instead seeking to regulate it. The FCC could improve broadband delivery by auctioning off much-needed spectrum. Or it could get rid of some of its own burdensome rules that prevent companies from attracting investors and innovating. But instead of doing these things that would improve the Internet, the FCC is wasting its time in an obsessive drive to regulate. This tells the American people that once again, a Washington agency is working in aself-interested way to increase its own power at the expense of the free people it is meant to serve.

### Docket 16-42

Here, we identify two very large sets of comments in this docket, by searching for the short phrases below. Please see the associated BuzzFeed News article for context.

The "American Commitment" set of comments is labeled as such because it uses language from [that organization's comment campaign](http://web.archive.org/web/20160403182941/https://www.americancommitment.org/cablebox-petition). (The text of comments appear to be generated algorithmically, selecting randomly from sets of pre-selected words and phrases, but the phrase used here for classification is static — it does not change across the comments.) To be sure, it is possible the comments were submitted by entities other than American Commitment, using the same language; the FCC's public portal does not specify who submitted these comments. 

In [21]:
comments_16_42_classified = (
    comments_16_42
    .pipe(
        add_classification,
        [
            ("cloud-based video on demand, and apps providing news", "American Commitment"),
            ("A cable subscriber pays over \$200", "'over $200'"),
        ]
    )
)

(
    comments_16_42_classified
    ["group"]
    .value_counts()
)

'over $200'            104816
American Commitment    101783
[other]                 75175
Name: group, dtype: int64

#### Dates submitted, by two main groups of comments

In [22]:
(
    comments_16_42_classified
    .loc[lambda df: df["group"] == "'over $200'"]
    ["date"]
    .value_counts()
    .sort_index()
)

2016-04-19    10499
2016-04-20    59247
2016-04-21    35070
Name: date, dtype: int64

In [23]:
(
    comments_16_42_classified
    .loc[lambda df: df["group"] == "American Commitment"]
    ["date"]
    .value_counts()
    .sort_index()
)

2016-02-22        1
2016-05-16    12293
2016-05-17    55852
2016-05-18    33637
Name: date, dtype: int64

*Note: The 2016-02-22 comment above appears to stem from a data-entry mistake on the FCC's website. There, [the comment](https://www.fcc.gov/ecfs/filing/60001484317)'s text seems to suggests that the language actually came from a [comment with ID 60001843102](https://www.fcc.gov/ecfs/filing/60001843102); that comment, in turn, says it was received on May 18, 2016.*

#### Example comments

In [24]:
print_example_comments(comments_16_42_classified)

=== 'over $200' ===

60001650840.txtA cable subscriber pays over $200 per year to rent a box from the cable companies that are already protected by government. This kills competition, limits consumer choice, and lifts up cable profits that are already excessive.Allow the free market to work and unlock the box to open competition and end the monopoly that cable companies have over our televisions.Page 1

60001633497.txtA cable subscriber pays over $200 per year to rent a box from the cable companies that are already protected by government. This kills competition, limits consumer choice, and lifts up cable profits that are already excessive.Allow the free market to work and unlock the box to open competition and end the monopoly that cable companies have over our televisions.Page 1

60001621406.txtA cable subscriber pays over $200 per year to rent a box from the cable companies that are already protected by government. This kills competition, limits consumer choice, and lifts up cable p

Compare the above timing and language to the [comment attributed to Annie Reeves](https://www.fcc.gov/ecfs/filing/60001803771), received by the FCC on May 17, 2016:

> The market for video content is booming and extremely competitive, offering a vast array of video streaming services, cloud-based video on demand, and apps providing news, cinema and programming. This market is swiftly innovating beyond the traditional set-top box to new applications and devices with more options than ever. Past Commission attempts to regulate set-top boxes have been a complete failure. Yet another failed attempt at heavy-handed government regulation will only stifle innovation and benefit companies with political influence rather than companies thatprovide what consumers want. We don't need the federal government to fix what isn'tbroken -- I urge you to reject the proposed rule.

# Examine email address crossover between 16-42 and bulk-uploaded 17-108 comments

Here, we calculate the proportion of commenters from docket 16-42 that later appeared in comments bulk-uploaded to docket 17-108, and observe a very high rate of overlap between the email addresses associated with comments that used American Commitment's language in docket 16-42 and the email addresses listed in comments bulk-uploaded by Media Bridge. We find the same for commenters' full names plus physical addresses.

In [25]:
(
    comments_16_42_classified
    [[
        "email_address",
        "name_and_location",
        "group",
    ]]
    .drop_duplicates()
    .dropna()
    .assign(
        email_isin_17_108_nonmb = lambda df: (
            df["email_address"].notnull() & df["email_address"].isin(
                bulk_uploads_17_108
                .loc[lambda df: df["uploader"] != "shane@mediabridgellc.com"]
                ["email_address"]
            )
        ),
        email_isin_17_108_mb = lambda df: (
            df["email_address"].notnull() & df["email_address"].isin(
                bulk_uploads_17_108
                .loc[lambda df: df["uploader"] == "shane@mediabridgellc.com"]
                ["email_address"]
            )
        ),
        name_and_location_isin_17_108_mb = lambda df: (
            df["name_and_location"].isin(
                bulk_uploads_17_108
                .loc[lambda df: df["uploader"] == "shane@mediabridgellc.com"]
                ["name_and_location"]
            )
        ),
    )
    .groupby("group")
    .pipe(lambda grp: pd.DataFrame({
        "num_emails": grp.size(),
        "email_isin_17_108_nonmb": grp["email_isin_17_108_nonmb"].mean().round(4),
        "email_isin_17_108_mb": grp["email_isin_17_108_mb"].mean().round(4),
        "name_and_location_isin_17_108_mb": grp["name_and_location_isin_17_108_mb"].mean().round(4),
    }))
    .loc[lambda df: df["num_emails"] >= 1000]
    .sort_values("email_isin_17_108_mb", ascending = False)
)

,num_emails,email_isin_17_108_nonmb,email_isin_17_108_mb,name_and_location_isin_17_108_mb
group,,,,
American Commitment,100252,0.0231,0.9987,0.9987
'over $200',100482,0.0243,0.0601,0.0566


# Analyze 17-108 bulk-uploads vis-a-vis Have I Been Pwned

In this section, we take random samples of email addresses the comments bulk-uploaded to Docket 17-108,  and calculate the rates at which they have appeared in the data breaches tracked by Have I Been Pwned. We focus on the accounts that uploaded comments containing 10,000+ distinct email addresses.

*Note: The HIBP data has already been been fetched and saved, but the code used to fetch the data is included here for reference, and for reuse by other researchers.*

In [26]:
try:
    uuid_lookup = pd.read_csv(
        BASE_PATH + "bulk-uploads-17-108-uuid-lookup.csv",
        dtype = str,
    )
    
    assert uuid_lookup["email_address_uuid"].value_counts().max() == 1
    print(f"{len(uuid_lookup):,d}")
except:
    uuid_lookup = pd.DataFrame(None, columns = [ "email_address", "email_address_uuid" ])

In [27]:
BASE_HIBP_URL = "https://haveibeenpwned.com/api/v3/breachedaccount/"
from json import JSONDecodeError

def fetch_hipb_results(email_address):
    while True:
        try:
            res = requests.get(
                f"{BASE_HIBP_URL}{email_address.strip()}",
                headers = {
                    'hibp-api-key': HIBP_KEY,
                },
            )
            if res.from_cache == False:
                time.sleep(1.5)

            # Check that JSON is parseable
            if res.content != b"":
                res.json()
                if "message" in res.json():
                    raise Exception("HIPB error: {res.json()['message']}")

        except requests.RequestException:
            sys.stderr.write(f"\nException; sleeping for 10 seconds\n")
            time.sleep(10)            
            continue
            
        except JSONDecodeError as e:
            sys.stderr.write(f"\nERROR: <{email_address}>\n")
            sys.stderr.write(f"{e}\n")
            sys.stderr.write(f"{res.content}\n")
            return [ { "email_address": email_address, "breach": "[error]" } ]

        if res.status_code == 429:
            sleep_int = int(res.headers["Retry-After"])
            sys.stderr.write(f"\nSleeping for {sleep_int + 1} seconds")
            time.sleep(sleep_int)
            continue
            
        if res.content == b"" or res.status_code == 404:
            return [ { "email_address": email_address, "breach": "[none]" } ]

        else:
            return [ { "email_address": email_address, "breach": x["Name"] } for x in res.json() ]

The following function creates a sample — or a grouped set of samples — from a given set of comments. Before sampling, the code removes blank email addresses and those with non-standard characters.

In [28]:
def create_sample(df, grouping = [], n = 1000, random_state = RANDOM_STATE):
    clean = (
        df
        .loc[lambda df: df["email_address"].notnull()]
        .loc[lambda df: df["email_address_nonstandard"] == 0]
        .drop_duplicates(subset = [ "email_address" ] + grouping)
    )
    
    sampler = lambda df: df.sample(n, random_state = random_state)
    
    if len(grouping):
        return (
            clean
            .groupby(grouping)
            .apply(sampler)
            .reset_index(drop = True)
        )
    else:
        return clean.pipe(sampler)

In [29]:
def get_breaches(df, save_path, save = True, use_saved = True):
    if use_saved and os.path.exists(save_path):
        return pd.read_csv(save_path, dtype = str)
    else:
        breaches_raw = pd.concat(map(pd.DataFrame, (
            df
            .rename(columns = {
                "email_address": "email_address_uuid"
            })
            .merge(
                uuid_lookup,
                how = "left",
                on = [ "email_address_uuid" ]
            )
            ["email_address"]
            .progress_apply(fetch_hipb_results)
        ))).drop_duplicates()
        
        breaches = (
            breaches_raw
            .merge(
                uuid_lookup,
                how = "left",
                on = [ "email_address" ]
            )
            .drop(columns = [ "email_address" ])
            .rename(columns = {
                "email_address_uuid": "email_address",
            })
        )
        
        if save:
            breaches.to_csv(save_path, index = False)

        return breaches

The function below calculates the breach rates for groups of sampled comments, for each breach found.

In [30]:
def calculate_breach_rates(sample, breaches):
    return (
        sample
        [[
            "email_address",
            "uploader",
        ]]
        
        .merge(
            breaches,
            how = "left",
            on = [ "email_address" ],
        )
        .assign(breached = 1)
        .set_index([
            "uploader",
            "email_address",
            "breach",
        ])
        ["breached"]
        .unstack()
        .fillna(0)
        .astype(int)
        # At this point, we have a matrix of uploader+email x breach
        # where the values are 1 if breached and 0 if not
        
        # Now, we group by uploader and calculate the proportion of
        # emails breached
        .groupby([ "uploader" ])
        .mean()
        
        # Then we return the data frame to a "tidy" format:
        # uploader|breach|rate
        .stack()
        .sort_values(ascending = False)
        .to_frame("rate")
        .reset_index()
    )

### 17-108 by bulk uploader

Limited here to the accounts that uploaded comments containing 10,000+ distinct email addresses.

In [31]:
sample_17_108_bulk_uploads = (
    bulk_uploads_17_108
    .loc[lambda df: df["uploader"].isin(
        uploader_metrics
        .loc[lambda df: df["unique_emails"] >= 10000]
        .index
    )]
    .pipe(
        create_sample,
        grouping = [ "uploader" ],
        n = 1000
    )
)

sample_17_108_bulk_uploads.head()

,date,comments,file,uploader,email_address_nonstandard,email_address,email_domain,name_and_location
0,5/15/17,"In 2015, wealthy leftist billionaires and powe...",FOI-14090-2017527.csv,esmisc@mac.com,0,939bfae2-62d1-47de-b009-c2abc6b681f5,yahoo.com,8930069a-021b-4263-9c3b-a3923af9a9dc
1,8/5/17,"Before leaving office, the Obama Administratio...",CFIF_1_25000_08052017_4 (1).csv,esmisc@mac.com,0,f9a12339-56cb-4540-9adc-fc6238428f49,gmail.com,6c65ec31-5135-4500-99c5-bba309b415fb
2,8/6/17,"Before leaving office, the Obama Administratio...",CFIF_1_25000_08062017_2.csv,esmisc@mac.com,0,fcf0991a-0ed7-408b-8e52-4735baccd906,yahoo.com,6dfa9546-ad61-404a-bec7-48464be021b4
3,7/29/17,"Before leaving office, the Obama Administratio...",CFIF_1_40000_07292017.csv,esmisc@mac.com,0,ee33e2a5-854f-471b-adb1-1ff62d69bf46,gmail.com,6d99eb3f-9242-440d-be2a-c7f7ae3b4e91
4,5/9/17,Obama's Federal Communications Commission (FCC...,T2017510-2.csv,esmisc@mac.com,0,4d294840-8365-4d34-a5c4-c09f6b8bc01d,icloud.com,d132203a-a146-4043-b097-d6606498309f


In [32]:
sample_17_108_bulk_uploads["uploader"].value_counts().sort_index()

esmisc@mac.com                1000
fccfreedom@hmamail.com        1000
karen@momsrising.org          1000
kurt@demandprogress.org       1000
meaghan@mandatemedia.com      1000
mike@fightforthefuture.org    1000
ncatalano@ofa.us              1000
shane@mediabridgellc.com      1000
Name: uploader, dtype: int64

In [33]:
breaches_17_108_bulk_uploads = (
    sample_17_108_bulk_uploads
    .pipe(get_breaches, "../data/breaches-17-108-bulk-uploads-sample.csv")
)

breaches_17_108_bulk_uploads.head()

,breach,email_address
0,8tracks,939bfae2-62d1-47de-b009-c2abc6b681f5
1,Animoto,939bfae2-62d1-47de-b009-c2abc6b681f5
2,MindJolt,939bfae2-62d1-47de-b009-c2abc6b681f5
3,ModernBusinessSolutions,939bfae2-62d1-47de-b009-c2abc6b681f5
4,RiverCityMedia,939bfae2-62d1-47de-b009-c2abc6b681f5


Most common breach-uploader combinations:

In [34]:
(
    calculate_breach_rates(
        sample_17_108_bulk_uploads,
        breaches_17_108_bulk_uploads,
    )
    .head(20)
)

,uploader,breach,rate
0,shane@mediabridgellc.com,ModernBusinessSolutions,0.942
1,shane@mediabridgellc.com,RiverCityMedia,0.807
2,fccfreedom@hmamail.com,VerificationsIO,0.782
3,shane@mediabridgellc.com,VerificationsIO,0.743
4,fccfreedom@hmamail.com,RiverCityMedia,0.645
5,esmisc@mac.com,VerificationsIO,0.625
6,esmisc@mac.com,RiverCityMedia,0.565
7,fccfreedom@hmamail.com,ModernBusinessSolutions,0.466
8,ncatalano@ofa.us,VerificationsIO,0.463
9,karen@momsrising.org,VerificationsIO,0.459


Modern Business Solutions breaches only:

In [35]:
(
    calculate_breach_rates(
        sample_17_108_bulk_uploads,
        breaches_17_108_bulk_uploads,
    )
    .loc[lambda df: df["breach"] == "ModernBusinessSolutions"]    
    .head(20)
)

,uploader,breach,rate
0,shane@mediabridgellc.com,ModernBusinessSolutions,0.942
7,fccfreedom@hmamail.com,ModernBusinessSolutions,0.466
16,esmisc@mac.com,ModernBusinessSolutions,0.345
116,meaghan@mandatemedia.com,ModernBusinessSolutions,0.114
121,ncatalano@ofa.us,ModernBusinessSolutions,0.106
130,karen@momsrising.org,ModernBusinessSolutions,0.099
145,kurt@demandprogress.org,ModernBusinessSolutions,0.087
150,mike@fightforthefuture.org,ModernBusinessSolutions,0.086


### Larger 17-108 Media Bridge sample (10,000 addresses), for more precise rates

In [36]:
sample_17_108_mb = (
    bulk_uploads_17_108
    .loc[lambda df: df["uploader"] == "shane@mediabridgellc.com"]
    .pipe(
        create_sample,
        n = 10000,
        random_state = RANDOM_STATE + 1,  # +1 so that we have an independent sample    
    )
)

sample_17_108_mb.head()

,date,comments,file,uploader,email_address_nonstandard,email_address,email_domain,name_and_location
3236533,5/14/17,"Dear Chairman Pai, I am concerned about Inter...",Batch-A4.csv,shane@mediabridgellc.com,0,44a8867c-3332-403f-9b34-560c054bd728,gmail.com,058cbe31-9c92-4509-a8d9-50f84a1cf1ae
2723370,5/15/17,"Dear Mr. Pai, Regarding the Obama takeover of...",Batch-A2.csv,shane@mediabridgellc.com,0,f9ad6e74-115c-4ccd-8a4e-f1e408423942,icloud.com,99c87532-c65e-42fa-97bd-0438d3ff504c
56337,5/14/17,"Chairman Pai: Hi, I'd like to comment on Titl...",file-i.csv,shane@mediabridgellc.com,0,ad374006-bbed-4c8b-932d-e7dfacce1a29,aol.com,6752ca9d-c848-4f3d-a66c-49073afe2458
592618,05/16/2017,The Title II order created a gaping gap in pri...,batch-d-4.csv,shane@mediabridgellc.com,0,c16e61f3-bd1f-4e50-95dd-830f5a219543,gmail.com,ebccbc49-fa14-404f-bf83-b0ef00d48e78
1014169,05/15/2017,"Dear Chairman Pai, I'm very worried about Net...",batch-b-5.csv,shane@mediabridgellc.com,0,b252ca16-b5a2-4c61-9034-6e365bec0beb,gmail.com,bc102bc2-d454-44a4-974d-d0b1a377f392


In [37]:
breaches_17_108_mb = (
    sample_17_108_mb
    .pipe(get_breaches, "../data/breaches-17-108-mb-sample.csv")
)

breaches_17_108_mb.head()

,breach,email_address
0,Edmodo,44a8867c-3332-403f-9b34-560c054bd728
1,ModernBusinessSolutions,44a8867c-3332-403f-9b34-560c054bd728
2,RiverCityMedia,44a8867c-3332-403f-9b34-560c054bd728
3,SpecialKSpamList,44a8867c-3332-403f-9b34-560c054bd728
4,VerificationsIO,44a8867c-3332-403f-9b34-560c054bd728


In [38]:
(
    calculate_breach_rates(
        sample_17_108_mb,
        breaches_17_108_mb,
    )
    .head(10)
)

,uploader,breach,rate
0,shane@mediabridgellc.com,ModernBusinessSolutions,0.9388
1,shane@mediabridgellc.com,RiverCityMedia,0.8277
2,shane@mediabridgellc.com,VerificationsIO,0.7651
3,shane@mediabridgellc.com,Collection1,0.2574
4,shane@mediabridgellc.com,Exactis,0.2571
5,shane@mediabridgellc.com,MySpace,0.1968
6,shane@mediabridgellc.com,AntiPublic,0.1956
7,shane@mediabridgellc.com,SpecialKSpamList,0.1946
8,shane@mediabridgellc.com,OnlinerSpambot,0.1941
9,shane@mediabridgellc.com,ExploitIn,0.1826


## Comparing MBS breach status to Docket 16-42 overlap

In [39]:
sample_17_108_mb_comparison = (
    sample_17_108_mb    
    .assign(
        isin_mbs = lambda df: (
            df
            ["email_address"].isin(
                breaches_17_108_mb
                .loc[lambda df: df["breach"] == "ModernBusinessSolutions"]
                ["email_address"]
            )
        ),
        isin_16 = lambda df: (
            df
            ["name_and_location"]
            .isin(
                comments_16_42_classified
                .loc[lambda df: df["group"] == "American Commitment"]
                ["name_and_location"]
            )
        )
    )
    [[
        "isin_mbs",
        "isin_16",
    ]]
)

sample_17_108_mb_comparison.head()

,isin_mbs,isin_16
3236533,True,False
2723370,True,False
56337,True,False
592618,True,False
1014169,True,False


Matrix of the 10,000-comment sample, by whether the email address exists in the Modern Business Solutions breach and whether the exact contact information shows up in the Docket 16-42 comments that used American Commitment's language:

In [40]:
(
    sample_17_108_mb_comparison
    .groupby([
        "isin_mbs",
        "isin_16",
    ])
    .size()
    .unstack()
)

isin_16,False,True
isin_mbs,,
False,35,577
True,9287,101


Among comments whose email addresses do *not* appear in MBS, this is the proportion that use exactly the same contact information as in the Docket 16-42 comments using American Commitment's language:

In [41]:
(
    sample_17_108_mb_comparison
    .loc[lambda df: df["isin_mbs"] == False]
    ["isin_16"]
    .mean()
    .round(4)
)

0.9428

Among comments whose email addresses use exactly the same contact information as in the Docket 16-42 comments using American Commitment's language, this is the proportion of email addresses that appear in MBS:

In [42]:
(
    sample_17_108_mb_comparison
    .loc[lambda df: df["isin_16"] == True]
    ["isin_mbs"]
    .mean()
    .round(4)
)

0.149

This is the proportion of comments that *either* are attributed to email addresses that appear in the Modern Business Solutions breach *or* use exactly the same contact information as in the Docket 16-42 comments using American Commitment's language:

In [43]:
(
    sample_17_108_mb_comparison
    [[
        "isin_mbs",
        "isin_16",
    ]]
    .sum(axis = 1)
    .pipe(lambda x: x > 0)
    .mean()
    .round(4)
)

0.9965

---

---

---